In [24]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import time
import datetime

In [15]:
import warnings
warnings.filterwarnings("ignore")

### Create supervised models

In [29]:
def create_models(models):
    ret = []
    for model in models:
        m = create_model(model)
        if m is not None:
            ret.append({'name': model, 'model': m})
    return ret
    
def create_model(model):
    if model == 'svm_linear':
        return svm_linear
    return None

def svm_linear():

    return LinearSVC()

## UIUC Benchmark

In [22]:
dataset = load_uiuc('pt')
create_features(['bow'], dataset['train'], dataset['test'])
models = create_models(['svm_linear'])
X = np.array([list(x) for x in dataset['train']['bow'].values])
y = dataset['train']['class'].values
X_ = np.array([list(x) for x in dataset['test']['bow'].values])
y_ = dataset['test']['class'].values

In [19]:
run_benchmark(models, X, y, X_, y_, sizes_train=[1000, 2000, 3000, 4000, 5500], save='results/en_svm_bow.csv')

  svm_linear .....
Run time benchmark: 0.865403413772583


,accuracy,confusion,datetime,execution_time,f1,model,precision,recall,test_time,train_size
0,0.782,"[[6, 0, 0, 0, 0, 0], [3, 134, 20, 5, 16, 24], ...",2019-04-01 16:04:54.847449,0.044162,0.786203,svm_linear,0.759875,0.849194,0.004571,1000
0,0.834,"[[7, 0, 0, 0, 0, 0], [2, 136, 15, 4, 10, 13], ...",2019-04-01 16:04:54.969716,0.105440,0.836288,svm_linear,0.815585,0.872081,0.004536,2000
0,0.856,"[[7, 0, 0, 0, 0, 0], [2, 136, 15, 2, 9, 10], [...",2019-04-01 16:04:55.133890,0.144723,0.857715,svm_linear,0.839093,0.889804,0.004657,3000
0,0.868,"[[7, 0, 0, 0, 0, 0], [2, 137, 17, 2, 7, 12], [...",2019-04-01 16:04:55.362341,0.208563,0.867945,svm_linear,0.849916,0.904584,0.004786,4000
0,0.874,"[[7, 0, 0, 0, 0, 0], [2, 137, 18, 2, 6, 9], [0...",2019-04-01 16:04:55.648261,0.266083,0.872246,svm_linear,0.853473,0.906299,0.004737,5500


In [21]:
run_benchmark(models, X, y, X_, y_, sizes_train=[1000, 2000, 3000, 4000, 5500], save='results/es_svm_bow.csv')

  svm_linear .....
Run time benchmark: 1.793912649154663


,accuracy,confusion,datetime,execution_time,f1,model,precision,recall,test_time,train_size
0,0.757576,"[[0, 21, 0, 0, 0, 0], [0, 123, 50, 13, 9, 30],...",2019-04-01 16:05:03.770018,0.051341,0.600857,svm_linear,0.600523,0.605281,0.025631,1000
0,0.763489,"[[0, 21, 0, 0, 0, 1], [0, 114, 46, 5, 8, 21], ...",2019-04-01 16:05:04.044172,0.225322,0.598812,svm_linear,0.598435,0.604866,0.025676,2000
0,0.778271,"[[0, 21, 0, 0, 1, 1], [0, 103, 36, 5, 5, 18], ...",2019-04-01 16:05:04.422774,0.327113,0.612952,svm_linear,0.613702,0.622098,0.025563,3000
0,0.779749,"[[0, 22, 0, 0, 0, 0], [0, 99, 36, 5, 10, 19], ...",2019-04-01 16:05:04.884717,0.410652,0.610892,svm_linear,0.611130,0.620432,0.025222,4000
0,0.777531,"[[0, 23, 0, 0, 0, 0], [0, 98, 33, 5, 10, 23], ...",2019-04-01 16:05:05.459525,0.521013,0.612686,svm_linear,0.611999,0.624477,0.026994,5500


In [23]:
run_benchmark(models, X, y, X_, y_, sizes_train=[1000, 2000, 3000, 4000, 5500], save='results/pt_svm_bow.csv')

  svm_linear .....
Run time benchmark: 0.8202826976776123


,accuracy,confusion,datetime,execution_time,f1,model,precision,recall,test_time,train_size
0,0.776,"[[3, 2, 0, 0, 0, 0], [6, 131, 25, 4, 15, 15], ...",2019-04-01 16:05:11.333671,0.039298,0.721988,svm_linear,0.698136,0.774230,0.004661,1000
0,0.816,"[[3, 0, 0, 0, 0, 0], [6, 130, 21, 4, 13, 6], [...",2019-04-01 16:05:11.451135,0.100402,0.762654,svm_linear,0.732438,0.858675,0.004661,2000
0,0.834,"[[3, 2, 0, 0, 0, 0], [6, 129, 16, 2, 8, 4], [0...",2019-04-01 16:05:11.613328,0.142826,0.769136,svm_linear,0.751979,0.803127,0.004639,3000
0,0.850,"[[4, 2, 0, 0, 0, 0], [5, 130, 18, 3, 6, 5], [0...",2019-04-01 16:05:11.815191,0.182008,0.800910,svm_linear,0.782601,0.831609,0.004887,4000
0,0.852,"[[5, 2, 0, 0, 0, 0], [4, 128, 16, 2, 7, 4], [0...",2019-04-01 16:05:12.094202,0.258467,0.815029,svm_linear,0.801578,0.835882,0.004998,5500


#### Load dataset

In [2]:
def load_uiuc(language):
    # language: 'en', 'pt', 'es'
    dataset = {
        'train': pd.read_csv('datasets/UIUC_' + language + '/train.csv'),
        'test': pd.read_csv('datasets/UIUC_' + language + '/test.csv'),
    }
    return dataset

#### Add Features

In [3]:
 # features: ['bow', ]
def create_features(features, df_train_, df_test_):
    for feature in features:
        create_feature(feature, df_train_, df_test_)
        
def create_feature(feature, df_train_, df_test_):
    if feature == 'bow':
        model = CountVectorizer(analyzer='word', strip_accents=None, 
                                ngram_range=(1, 1), lowercase=True, 
                                max_features=5000)
        model.fit(pd.concat([df_train_['question'], df_test_['question']]))
        ret = model.transform(df_train_['question']).toarray()
        df_train_['bow'] = [x for x in ret]
        ret = model.transform(df_test_['question']).toarray()
        df_test_['bow'] = [x for x in ret]

#### Benchmark

In [5]:
def run_benchmark(models, X, y, x_test, y_test, save='default.csv', sizes_train=[], start_results=None, metric_average="macro"):
    
    start_benchmark = time.time()
    
    last_model = None
    last_train_size = None
    results = pd.DataFrame()
    if start_results is not None:
        last_model = start_results['model'].tail(1)
        last_size_train = start_results['size_train'].tail(1)
        results = start_results
    for model in models:
            
        print(' ', model['name'], end=' ')
            
        for size_train in sizes_train:
            
            if start_results is not None:
                if model in start_results['model'].unique() and last_size_train >= size_train:
                        continue
            start_results = None
            
            print('.', end='')

            x_train = X[:size_train]
            y_train = y[:size_train]

            m = model['model']()
            start_time = time.time()
            m.fit(x_train, y_train)
            train_time = time.time() - start_time

            start_time = time.time()
            result = m.predict(x_test)
            test_time = time.time() - start_time
            
            data = {'datetime': datetime.datetime.now(),
                          'model': model['name'],
                          'accuracy': accuracy_score(result, y_test),
                          'precision': precision_score(result, y_test, average=metric_average),
                          'recall': recall_score(result, y_test, average=metric_average),
                          'f1': f1_score(result, y_test, average=metric_average),
                          'confusion': confusion_matrix(result, y_test),
                          'train_size': size_train,
                          'execution_time': train_time,
                          'test_time': test_time}
            results = results.append([data])
            results.to_csv(save)
    print('')
    aux = time.time() - start_benchmark
    print('Run time benchmark:', aux)
    return pd.DataFrame(results)

## DISEQuA Benchmark

In [57]:
dataset = load_disequa('SPA')
create_features(['bow'], dataset)
models = create_models(['svm_linear'])
X = np.array([list(x) for x in dataset['bow'].values])
y = dataset['class'].values

In [52]:
run_benchmark(models, X, y, sizes_train=[100,200,300,400], save='results/DISEQuA_DUT_svm_bow.csv')

  svm_linear ........................................
Run time benchmark: 0.8952779769897461


,accuracy,confusion,datetime,execution_time,f1,fold,model,precision,recall,test_time,train_size
0,0.674286,"[[42, 0, 0, 0, 0, 0, 3], [2, 49, 2, 5, 13, 17,...",2019-04-01 16:47:13.481287,0.001603,0.532134,0,svm_linear,0.532851,0.550911,0.008804,100
0,0.682857,"[[44, 4, 3, 1, 0, 1, 3], [4, 50, 4, 4, 5, 4, 7...",2019-04-01 16:47:13.505505,0.002148,0.574254,1,svm_linear,0.576236,0.611425,0.001610,100
0,0.634286,"[[38, 2, 1, 5, 0, 5, 4], [5, 53, 1, 3, 13, 15,...",2019-04-01 16:47:13.532340,0.002130,0.493544,2,svm_linear,0.512946,0.520656,0.002462,100
0,0.634286,"[[44, 0, 2, 0, 0, 2, 2], [0, 49, 2, 3, 17, 14,...",2019-04-01 16:47:13.550687,0.001996,0.511407,3,svm_linear,0.513014,0.556371,0.000773,100
0,0.637143,"[[38, 2, 1, 1, 1, 5, 5], [3, 56, 3, 6, 8, 14, ...",2019-04-01 16:47:13.568796,0.002000,0.473604,4,svm_linear,0.499112,0.487689,0.000746,100
0,0.682857,"[[43, 3, 1, 0, 3, 1, 1], [2, 55, 0, 3, 9, 14, ...",2019-04-01 16:47:13.587181,0.002025,0.521634,5,svm_linear,0.544385,0.526651,0.000753,100
0,0.668571,"[[40, 5, 1, 1, 0, 0, 0], [6, 53, 4, 4, 18, 17,...",2019-04-01 16:47:13.605435,0.001950,0.537473,6,svm_linear,0.540242,0.561972,0.000758,100
0,0.671429,"[[35, 2, 0, 1, 1, 4, 0], [4, 55, 3, 8, 13, 11,...",2019-04-01 16:47:13.624123,0.002185,0.501328,7,svm_linear,0.528289,0.501407,0.000756,100
0,0.674286,"[[42, 3, 1, 0, 0, 1, 1], [0, 50, 1, 2, 2, 5, 3...",2019-04-01 16:47:13.644148,0.003129,0.542487,8,svm_linear,0.541535,0.563350,0.000807,100
0,0.634286,"[[36, 2, 0, 1, 0, 1, 0], [7, 51, 8, 7, 8, 15, ...",2019-04-01 16:47:13.663270,0.002109,0.531333,9,svm_linear,0.533356,0.578033,0.000756,100


In [54]:
run_benchmark(models, X, y, sizes_train=[100,200,300,400], save='results/DISEQuA_ENG_svm_bow.csv')

  svm_linear ........................................
Run time benchmark: 0.82710862159729


,accuracy,confusion,datetime,execution_time,f1,fold,model,precision,recall,test_time,train_size
0,0.717143,"[[49, 1, 0, 1, 0, 3, 4], [1, 49, 4, 0, 7, 4, 6...",2019-04-01 16:47:36.874204,0.001744,0.571127,0,svm_linear,0.578213,0.571679,0.010116,100
0,0.757143,"[[47, 2, 2, 1, 1, 1, 1], [0, 55, 2, 1, 3, 1, 5...",2019-04-01 16:47:36.890462,0.002356,0.653680,1,svm_linear,0.655708,0.786277,0.000965,100
0,0.697143,"[[38, 2, 1, 1, 2, 5, 0], [4, 50, 5, 3, 11, 8, ...",2019-04-01 16:47:36.909416,0.002187,0.552969,2,svm_linear,0.565220,0.563672,0.001542,100
0,0.688571,"[[43, 2, 1, 3, 1, 3, 2], [4, 50, 8, 3, 9, 10, ...",2019-04-01 16:47:36.927337,0.002098,0.581834,3,svm_linear,0.583011,0.636319,0.000745,100
0,0.708571,"[[39, 2, 2, 2, 1, 1, 2], [5, 55, 3, 3, 8, 13, ...",2019-04-01 16:47:36.945864,0.002044,0.561055,4,svm_linear,0.577145,0.567305,0.000980,100
0,0.714286,"[[44, 2, 2, 0, 0, 5, 0], [2, 60, 2, 3, 7, 8, 6...",2019-04-01 16:47:36.964294,0.002165,0.552277,5,svm_linear,0.577495,0.553236,0.000753,100
0,0.662857,"[[38, 6, 2, 1, 0, 8, 1], [7, 53, 6, 4, 13, 17,...",2019-04-01 16:47:36.982575,0.002016,0.541059,6,svm_linear,0.556005,0.607467,0.000752,100
0,0.688571,"[[38, 1, 1, 0, 2, 8, 2], [4, 57, 7, 4, 9, 18, ...",2019-04-01 16:47:37.001723,0.002553,0.550219,7,svm_linear,0.565477,0.637235,0.000755,100
0,0.722857,"[[44, 1, 1, 0, 0, 2, 0], [4, 57, 3, 1, 4, 2, 6...",2019-04-01 16:47:37.020738,0.002317,0.584967,8,svm_linear,0.589230,0.589002,0.000752,100
0,0.665714,"[[38, 1, 2, 1, 1, 0, 0], [7, 51, 5, 4, 5, 16, ...",2019-04-01 16:47:37.040698,0.002758,0.530555,9,svm_linear,0.549029,0.562169,0.000808,100


In [56]:
run_benchmark(models, X, y, sizes_train=[100,200,300,400], save='results/DISEQuA_ITA_svm_bow.csv')

  svm_linear ........................................
Run time benchmark: 0.8515074253082275


,accuracy,confusion,datetime,execution_time,f1,fold,model,precision,recall,test_time,train_size
0,0.691429,"[[45, 2, 2, 0, 2, 1, 1], [4, 43, 2, 4, 4, 7, 6...",2019-04-01 16:47:53.176738,0.001702,0.559593,0,svm_linear,0.561285,0.575779,0.001020,100
0,0.745714,"[[49, 5, 2, 0, 2, 1, 1], [1, 58, 4, 4, 3, 7, 1...",2019-04-01 16:47:53.201137,0.002517,0.610590,1,svm_linear,0.621415,0.630955,0.001811,100
0,0.691429,"[[42, 0, 1, 0, 2, 1, 0], [1, 53, 2, 7, 10, 14,...",2019-04-01 16:47:53.219166,0.001931,0.535222,2,svm_linear,0.554612,0.565453,0.000839,100
0,0.668571,"[[43, 2, 3, 0, 4, 1, 0], [5, 44, 4, 2, 4, 10, ...",2019-04-01 16:47:53.237224,0.002122,0.551954,3,svm_linear,0.558275,0.569646,0.000842,100
0,0.668571,"[[39, 6, 2, 1, 2, 2, 0], [9, 51, 6, 5, 8, 17, ...",2019-04-01 16:47:53.255416,0.002098,0.537525,4,svm_linear,0.544320,0.614947,0.000808,100
0,0.708571,"[[46, 4, 1, 1, 1, 4, 0], [1, 55, 3, 2, 6, 10, ...",2019-04-01 16:47:53.273822,0.001892,0.565713,5,svm_linear,0.577459,0.618334,0.000821,100
0,0.642857,"[[38, 4, 2, 1, 11, 7, 0], [4, 50, 5, 4, 8, 10,...",2019-04-01 16:47:53.292222,0.001854,0.496147,6,svm_linear,0.511516,0.507116,0.000831,100
0,0.671429,"[[41, 4, 3, 0, 4, 3, 1], [2, 52, 4, 5, 7, 18, ...",2019-04-01 16:47:53.310758,0.001832,0.495968,7,svm_linear,0.530366,0.595981,0.000830,100
0,0.700000,"[[43, 1, 1, 0, 0, 1, 0], [3, 53, 4, 3, 0, 6, 2...",2019-04-01 16:47:53.329449,0.001862,0.582804,8,svm_linear,0.580091,0.594233,0.000810,100
0,0.668571,"[[42, 1, 3, 0, 2, 1, 0], [5, 50, 5, 6, 3, 19, ...",2019-04-01 16:47:53.349940,0.002121,0.527966,9,svm_linear,0.549886,0.545992,0.000850,100


In [58]:
run_benchmark(models, X, y, sizes_train=[100,200,300,400], save='results/DISEQuA_SPA_svm_bow.csv')

  svm_linear ........................................
Run time benchmark: 0.934882402420044


,accuracy,confusion,datetime,execution_time,f1,fold,model,precision,recall,test_time,train_size
0,0.657143,"[[46, 0, 2, 0, 1, 3, 0], [1, 44, 3, 2, 14, 3, ...",2019-04-01 16:48:17.545280,0.001776,0.542274,0,svm_linear,0.546513,0.556039,0.000975,100
0,0.705714,"[[41, 2, 0, 1, 0, 2, 0], [2, 55, 2, 2, 7, 1, 3...",2019-04-01 16:48:17.563676,0.003320,0.589860,1,svm_linear,0.599195,0.644547,0.002221,100
0,0.671429,"[[42, 2, 0, 1, 2, 1, 1], [2, 52, 6, 5, 2, 12, ...",2019-04-01 16:48:17.585238,0.002028,0.521484,2,svm_linear,0.540285,0.542897,0.000823,100
0,0.654286,"[[46, 5, 1, 3, 2, 2, 1], [1, 43, 9, 2, 14, 12,...",2019-04-01 16:48:17.603362,0.002170,0.543918,3,svm_linear,0.545848,0.570056,0.000826,100
0,0.628571,"[[37, 2, 1, 2, 0, 6, 0], [7, 52, 6, 1, 10, 9, ...",2019-04-01 16:48:17.621809,0.002241,0.479606,4,svm_linear,0.494162,0.482328,0.000792,100
0,0.682857,"[[43, 4, 3, 0, 1, 2, 0], [4, 55, 3, 3, 7, 8, 6...",2019-04-01 16:48:17.640181,0.001922,0.517676,5,svm_linear,0.552920,0.514037,0.000820,100
0,0.657143,"[[37, 1, 0, 1, 3, 2, 0], [7, 54, 7, 2, 13, 8, ...",2019-04-01 16:48:17.658838,0.002183,0.550974,6,svm_linear,0.554161,0.573256,0.000791,100
0,0.625714,"[[38, 1, 2, 2, 2, 3, 0], [1, 52, 8, 4, 11, 16,...",2019-04-01 16:48:17.677682,0.002109,0.482105,7,svm_linear,0.505189,0.510820,0.000803,100
0,0.665714,"[[40, 1, 1, 1, 0, 1, 0], [1, 51, 2, 1, 0, 3, 2...",2019-04-01 16:48:17.696717,0.002219,0.565693,8,svm_linear,0.564638,0.577969,0.000796,100
0,0.648571,"[[42, 0, 3, 0, 2, 2, 2], [4, 48, 14, 5, 3, 19,...",2019-04-01 16:48:17.717520,0.002247,0.523271,9,svm_linear,0.559410,0.544125,0.000811,100


#### Load dataset

In [36]:
def load_disequa(language):
    df = pd.read_csv('datasets/DISEQuA/disequa.csv')
    return df[df['language'] == language]

#### Add features

In [31]:
# features: ['bow', ]
def create_features(features, df):
    for feature in features:
        create_feature(feature, df)
        
def create_feature(feature, df):
    if feature == 'bow':
        model = CountVectorizer(analyzer='word', strip_accents=None, 
                                ngram_range=(1, 1), lowercase=True, 
                                max_features=5000)
        model.fit(df['question'])
        ret = model.transform(df['question']).toarray()
        df['bow'] = [x for x in ret]

#### Benchmark

In [49]:
def run_benchmark(models, X, y, folds=10, seed=0, save='default.csv', sizes_train=[], start_results=None, metric_average="macro"):
    
    start_benchmark = time.time()
    
    last_model = None
    last_train_size = None
    last_fold = None
    results = pd.DataFrame()
    if start_results is not None:
        last_model = start_results['model'].tail(1)
        last_size_train = start_results['size_train'].tail(1)
        last_fold = start_results['fold'].tail(1)
        results = start_results
    
    
    
    for model in models:
            
        print(' ', model['name'], end=' ')
            
        for size_train in sizes_train:
            
            size_test = len(X) - size_train
            rs = ShuffleSplit(n_splits=folds, train_size=size_train, test_size=size_test, 
                          random_state=seed)
            fold = 0
            for train_indexs, test_indexs in rs.split(X):
            
                if start_results is not None:
                    if model in start_results['model'].unique() and last_size_train >= size_train and last_fold >= fold:
                        continue
                start_results = None

                print('.', end='')

                x_train = X[train_indexs]
                y_train = y[train_indexs]
                x_test = X[test_indexs]
                y_test = y[test_indexs]

                m = model['model']()
                start_time = time.time()
                m.fit(x_train, y_train)
                train_time = time.time() - start_time

                start_time = time.time()
                result = m.predict(x_test)
                test_time = time.time() - start_time

                data = {'datetime': datetime.datetime.now(),
                        'model': model['name'],
                        'accuracy': accuracy_score(result, y_test),
                        'precision': precision_score(result, y_test, average=metric_average),
                        'recall': recall_score(result, y_test, average=metric_average),
                        'f1': f1_score(result, y_test, average=metric_average),
                        'confusion': confusion_matrix(result, y_test),
                        'train_size': size_train,
                        'fold': fold,
                        'execution_time': train_time,
                        'test_time': test_time}
                results = results.append([data])
                results.to_csv(save)
                fold += 1
    print('')
    aux = time.time() - start_benchmark
    print('Run time benchmark:', aux)
    return pd.DataFrame(results)